# 可组合基础策略库
本教程将展示如何重用作为backtesting.py软件分发一部分的可组合基础交易策略。从此以后，假设您已经熟悉基本的软件包用法
[basic package usage](https://kernc.github.io/backtesting.py/doc/examples/Quick Start User Guide.html).

我们将扩展与《快速入门用户指南》中相同的移动平均线交叉策略，但我们会将其重写为矢量化信号策略并添加追踪止损。
同样，我们将使用我们的帮助程序移动平均函数。
[Quick Start User Guide](https://kernc.github.io/backtesting.py/doc/examples/Quick Start User Guide.html),



In [1]:
from backtesting.test import SMA

C:\ProgramData\Anaconda3\lib\site-packages\backtesting\_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

该软件发行版的一部分是backtesting.lib
[backtesting.lib](https://kernc.github.io/backtesting.py/doc/backtesting/lib.html)
模块，其中包含用于策略开发的各种可重用实用程序。其中一些实用程序是我们可以扩展和构建的可组合基础策略。
我们在这里导入并扩展了其中两个策略：

我们在这里导入并扩展了其中两个策略：
* [SignalStrategy](https://kernc.github.io/backtesting.py/doc/backtesting/lib.html#backtesting.lib.SignalStrategy)
  它决定单个信号向量是否买入头寸，类似于矢量化回溯测试引擎，以及
  [vectorized backtesting](https://www.google.com/search?q=vectorized+backtesting)

* [TrailingStrategy](https://kernc.github.io/backtesting.py/doc/backtesting/lib.html#backtesting.lib.TrailingStrategy)
  它会自动跟踪当前价格，止损订单是平均真实范围（ATR）的倍数。
  [average true range](https://en.wikipedia.org/wiki/Average_true_range)

In [12]:
import pandas as pd
from backtesting.lib import SignalStrategy, TrailingStrategy


class SmaCross(SignalStrategy,
               TrailingStrategy):
    n1 = 10
    n2 = 25
    
    def init(self):
        # 在init()和next()中，调用超方法来正确初始化父类是很重要的
        super().init()
        
        # 预先计算两个移动平均线
        sma1 = self.I(SMA, self.data.Close, self.n1)
        sma2 = self.I(SMA, self.data.Close, self.n2)
        
        # sma1和sma2向上交叉。Diff给出[-1,0，*1*]
        signal = (pd.Series(sma1) > sma2).astype(int).diff().fillna(0)
        # signal = signal.replace(-1, 0)  # 向上/long 只有
        
        # 在每个订单上使用95%的可用流动性(当时)。(保留值为1。而是只买一股。)
        entry_size = signal * .95
                
        # 使用' SignalStrategy '提供的方法设置订单输入大小。看文档。
        self.set_signal(entry_size=entry_size)
        
        # 使用' TrailingStrategy '提供的方法将跟踪止损设置为2倍ATR
        self.set_trailing_sl(2)

请注意，由于 中的策略可能需要自己的初始化和下一个价格变动逻辑，因此请确保始终在重写的方法中调用super（）.init（）和super（）.next（）。 lib

让我们看看示例策略在历史Google数据上的表现如何。

In [10]:
from backtesting import Backtest
from backtesting.test import GOOG

bt = Backtest(GOOG, SmaCross, commission=.002)

bt.run()


Start                     2004-08-19 00:00:00
End                       2013-03-01 00:00:00
Duration                   3116 days 00:00:00
Exposure Time [%]                   49.860335
Equity Final [$]                  24940.14508
Equity Peak [$]                   33203.90694
Return [%]                         149.401451
Buy & Hold Return [%]              703.458242
Return (Ann.) [%]                   11.317531
Volatility (Ann.) [%]               23.631493
Sharpe Ratio                         0.478917
Sortino Ratio                        0.841741
Calmar Ratio                         0.387229
Max. Drawdown [%]                  -29.226947
Avg. Drawdown [%]                   -6.149281
Max. Drawdown Duration      802 days 00:00:00
Avg. Drawdown Duration       78 days 00:00:00
# Trades                                   75
Win Rate [%]                        50.666667
Best Trade [%]                      48.410457
Worst Trade [%]                    -16.896792
Avg. Trade [%]                    

In [11]:
bt.plot()

Row(id='3203', ...)

请注意，使用追踪止损管理风险如何确保我们的收益并限制我们的损失。

有关此类其他策略以及其他一般可重用的实用程序，请参阅backtesting.lib模块参考。

通过探索更多示例了解更多信息，或在完整的 API 参考中找到更多框架选项。

[**_backtesting.lib_ module reference**](https://kernc.github.io/backtesting.py/doc/backtesting/lib.html).

Learn more by exploring further
[examples](https://kernc.github.io/backtesting.py/doc/backtesting/index.html#tutorials)
or find more framework options in the
[full API reference](https://kernc.github.io/backtesting.py/doc/backtesting/index.html#header-submodules).